In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from server.database import engine, create_all, drop_all
from sqlalchemy import insert, select
from server.database.tables import user_table, login_history_table

drop_all()
create_all()

2021-09-29 01:08:59,520 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:08:59,521 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-09-29 01:08:59,521 INFO sqlalchemy.engine.Engine [cached since 13.56s ago] {'table_schema': 'facerecognition', 'table_name': 'user'}
2021-09-29 01:08:59,523 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-09-29 01:08:59,524 INFO sqlalchemy.engine.Engine [cached since 13.56s ago] {'table_schema': 'facerecognition', 'table_name': 'login_history'}
2021-09-29 01:08:59,525 INFO sqlalchemy.engine.Engine 
DROP TABLE login_history
2021-09-29 01:08:59,525 INFO sqlalchemy.engine.Engine [no key 0.00041s] {}
2021-09-29 01:08:59,605 INFO sqlalchemy.engine.Engine 
DROP TABLE user
2021-09-29 01:08:59,605 INFO sqlalchemy.engine.Engine [no key 0.00067s] {}
2

In [4]:
from sqlalchemy import text

# https://docs.sqlalchemy.org/en/14/tutorial/dbapi_transactions.html

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2021-09-29 01:09:13,519 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:09:13,519 INFO sqlalchemy.engine.Engine select 'hello world'
2021-09-29 01:09:13,520 INFO sqlalchemy.engine.Engine [generated in 0.00149s] {}
[('hello world',)]
2021-09-29 01:09:13,520 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
with engine.connect() as conn:
    result = conn.execute(insert(user_table),
    [
        {'name': 'Kittie'},
        {'name': 'Lucas'},
        {'name': 'Kelly'},
        {'name': 'Ivan'},
    ])
    conn.commit()

2021-09-29 01:09:15,533 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:09:15,534 INFO sqlalchemy.engine.Engine INSERT INTO user (name) VALUES (%(name)s)
2021-09-29 01:09:15,534 INFO sqlalchemy.engine.Engine [generated in 0.00144s] ({'name': 'Kittie'}, {'name': 'Lucas'}, {'name': 'Kelly'}, {'name': 'Ivan'})
2021-09-29 01:09:15,545 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
with engine.connect() as conn:
    result = conn.execute(select(user_table))
    for row in result.all():
        print(row)

2021-09-29 01:09:46,418 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:09:46,419 INFO sqlalchemy.engine.Engine SELECT user.user_id, user.name, user.password, user.joined_date 
FROM user
2021-09-29 01:09:46,419 INFO sqlalchemy.engine.Engine [cached since 27.94s ago] {}
(1, 'Kittie', None, datetime.datetime(2021, 9, 29, 1, 9, 15))
(2, 'Lucas', None, datetime.datetime(2021, 9, 29, 1, 9, 15))
(3, 'Kelly', None, datetime.datetime(2021, 9, 29, 1, 9, 15))
(4, 'Ivan', None, datetime.datetime(2021, 9, 29, 1, 9, 15))
2021-09-29 01:09:46,420 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
import time

with engine.connect() as conn:
    for _ in range(5):
        result = conn.execute(insert(login_history_table),
        [
            {'user_id': 1, 'confidence': 60},
        ])
        conn.commit()
        time.sleep(1)


2021-09-29 01:11:35,431 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:11:35,432 INFO sqlalchemy.engine.Engine INSERT INTO login_history (user_id, confidence) VALUES (%(user_id)s, %(confidence)s)
2021-09-29 01:11:35,432 INFO sqlalchemy.engine.Engine [generated in 0.00166s] {'user_id': 1, 'confidence': 60}
2021-09-29 01:11:35,443 INFO sqlalchemy.engine.Engine COMMIT
2021-09-29 01:11:36,465 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:11:36,466 INFO sqlalchemy.engine.Engine INSERT INTO login_history (user_id, confidence) VALUES (%(user_id)s, %(confidence)s)
2021-09-29 01:11:36,466 INFO sqlalchemy.engine.Engine [cached since 1.036s ago] {'user_id': 1, 'confidence': 60}
2021-09-29 01:11:36,467 INFO sqlalchemy.engine.Engine COMMIT
2021-09-29 01:11:37,488 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 01:11:37,489 INFO sqlalchemy.engine.Engine INSERT INTO login_history (user_id, confidence) VALUES (%(user_id)s, %(confidence)s)
2021-09-29 01:11:37,489 